In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": 0},
    attn_implementation="sdpa",
)
# model = model.eval()
model.requires_grad_(False)

from datasets import load_dataset

dataset = load_dataset(
    # "dim/hendrycks_math_train_12k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    # "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

dataset = dataset["train"].train_test_split(
    # test_size=250,
    test_size=350,
    # test_size=999,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)

correct_dataset = []

for pos, item in enumerate(dataset):
    try:
        answer = dataset_answer_filter(item["answer"])
        model_answer = model_answer_filter(item["model_answer"])
        # print(answer, model_answer)
        # break
        if is_equiv(answer, model_answer):
            correct_dataset.append(item)
    except:
        pass

print(len(dataset), len(correct_dataset), len(correct_dataset) / len(dataset))

correct_dataset = correct_dataset[:30]
len(correct_dataset)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
224 202 0.9017857142857143


30

## test time train generation (single train)

In [ ]:
import torch

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
from tqdm.notebook import tqdm
from hidden_capacity_reasoning.utils import tokenize_single_turn

correct_items = 0
torch.manual_seed(0)
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

dataset_pos = 5
for dataset_pos in range(len(correct_dataset)):
    tokenized_turn = tokenize_single_turn(
        question=base_prompt.format(question=correct_dataset[dataset_pos]["problem"]),
        answer=correct_dataset[dataset_pos]["model_answer"],
        tokenizer=tokenizer,
    )
    for key in tokenized_turn.keys():
        tokenized_turn[key] = torch.tensor(tokenized_turn[key])

    evaluation_dataset = []

    input_ids = correct_dataset[dataset_pos]["problem"]

    input_ids = [
        tokenizer.apply_chat_template(
            [
                {
                    "role": "user",
                    "content": base_prompt.format(question=input_ids),
                },
            ],
            tokenize=True,
            add_generation_prompt=True,
        )
    ]

    device = "cuda"

    content_compression_mask = torch.tensor(tokenized_turn["content_compression_mask"])

    input_part_end = (content_compression_mask == 0).nonzero()[-3][0]
    question_input_ids = (
        tokenized_turn["input_ids"][: int(input_part_end) + 1].unsqueeze(0).cuda()
    )
    print(tokenizer.decode(question_input_ids[-1]))

    with torch.no_grad():

        # input_ids = torch.tensor(question_input_ids).cuda()
        input_ids_embeds = model.get_input_embeddings()(question_input_ids)
        max_new_tokens = 300

        inputs_embeds = torch.cat(
            [
                input_ids_embeds,
            ],
            dim=1,
        )
        generated_ids_new = model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=torch.ones(
                inputs_embeds.shape[:2],
                device="cuda",
            ).long(),
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )
        # break
    generated_result = tokenizer.decode(generated_ids_new[-1])
    print(generated_result)
    # get original language loss
    labels = torch.cat([question_input_ids.cuda(), generated_ids_new.cuda()], dim=1)
    print(tokenizer.decode(labels[-1]))

    question_content_mask = content_compression_mask[: int(input_part_end) + 1].clone()
    question_content_mask[question_content_mask == 0] = 4
    question_content_mask[question_content_mask == 1] = 0
    question_content_mask[question_content_mask == 4] = 1
    train_content_mask_new = torch.cat(
        [
            question_content_mask,
            torch.ones(
                generated_ids_new.shape[1],
            ),
        ]
    ).long()
    print(question_content_mask)

    generated_embeds = model.get_input_embeddings()(generated_ids_new)
    new_input_embeds = torch.cat(
        [
            input_ids_embeds,
            generated_embeds,
        ],
        dim=1,
    )
    labels[:, train_content_mask_new == 0] = -100

    with torch.no_grad():
        original_loss = model(
            inputs_embeds=new_input_embeds,
            # attention_mask=train_attention_mask.unsqueeze(0).cuda(),
            # attention_mask=torch.ones(new_input_embeds.size()[:2]).cuda().long(),
            labels=labels,
        ).loss
    original_loss
    break

/tmp/ipykernel_1214684/2285348861.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  content_compression_mask = torch.tensor(tokenized_turn["content_compression_mask"])
/home/user-name-goes-here/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/user-name-goes-here/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `e

<｜begin▁of▁sentence｜><｜User｜>Problem: Below is a magic square, meaning that the sum of the numbers in each row, in each column, and in each of the $2$ main diagonals are equal. What is the value of $n$?

[asy]size(125);
for(int i = 0; i<4; ++i)
{

draw((0,i)--(3,i),linewidth(1));
}

for(int j = 0; j<4; ++j)
{

draw((j,0)--(j,3),linewidth(1));
}

label("$n-3$",(.5,.5));
label("3",(.5,1.5));
label("$n+1$",(.5,2.5));

label("$n+2$",(1.5,.5));
label("$2n-9$",(1.5,1.5));
label("$1$",(1.5,2.5));

label("$2$",(2.5,.5));
label("$n$",(2.5,1.5));
label("$n-1$",(2.5,2.5));
[/asy]

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>

Okay, so I have this magic square problem here, and I need to find the value of \( n \). Let me try to figure this out step by step. 

First, I remember that a magic square is a grid where the sums of numbers in each row, each column, and both main diagonals are equal. That common sum is called the magic constant. So, my goal is 

In [6]:
labels = torch.cat([question_input_ids.cuda(), generated_ids_new.cuda()], dim=1)
print(tokenizer.decode(labels[-1]))

question_content_mask = content_compression_mask[: int(input_part_end) + 1].clone()
question_content_mask[question_content_mask == 0] = 4
question_content_mask[question_content_mask == 1] = 0
question_content_mask[question_content_mask == 4] = 1
train_content_mask_new = torch.cat(
    [
        question_content_mask,
        torch.ones(
            generated_ids_new.shape[1],
        ),
    ]
).long()
print(question_content_mask)

with torch.no_grad():
    generated_embeds = model.get_input_embeddings()(generated_ids_new)
    new_input_embeds = torch.cat(
        [
            input_ids_embeds,
            generated_embeds,
        ],
        dim=1,
    )
labels[:, train_content_mask_new == 0] = -100
# new_input_embeds
labels

<｜begin▁of▁sentence｜><｜User｜>Problem: In right triangle $ABC$ with $\angle B = 90^\circ$, we have $\sin A = 2\cos A$.  What is $\tan A$?

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>
Okay, so I have this problem here: In right triangle ABC with angle B equal to 90 degrees, we know that sin A equals 2 cos A. The question is asking for tan A. Hmm, let me think about how to approach this.

First, I remember that in a right triangle, the sine of an angle is the ratio of the opposite side to the hypotenuse, and the cosine is the ratio of the adjacent side to the hypotenuse. So, if angle A is one of the non-right angles in triangle ABC, then side opposite to angle A is BC, the side adjacent is AC, and the hypotenuse is AB.

Given that sin A = 2 cos A, I can write that as:

sin A = 2 cos A

I also know that tan A is sin A over cos A, so tan A = sin A / cos A. Since sin A is equal to 2 cos A, substituting that into the tan A equation gives:

tan A 

tensor([[151646, 151644,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100, 151645, 151648,    198,  32313,     11,    773,    358,
            614,    419,   3491,   1588,     25,    758,   1290,  21495,  19360,
            448,   9210,    425,   6144,    311,    220,     24,     15,  12348,
             11,    582,   1414,    429,   7437,    362,  16819,    220,     17,
           7960,    362,     13,    576,   3405,    374,  10161,    369,  14197,
            362,     13,  88190,     11,   1077,    752,   1744,    911,   1246,
            311,   5486,    

In [37]:
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
end_think_id = torch.tensor(tokenizer.encode("</think>", add_special_tokens=False)[0])
start_think_id = torch.tensor(
    tokenizer.encode("<｜Assistant｜><think>\n", add_special_tokens=False)
)
user_id = torch.tensor(tokenizer.encode("<｜User｜>", add_special_tokens=False)[0])

# train_attention_mask = (
#     input_ids == bos_id | input_ids == start_think_id | input_ids == user_id
# )

In [7]:
with torch.no_grad():
    original_loss = model(
        inputs_embeds=new_input_embeds,
        # attention_mask=train_attention_mask.unsqueeze(0).cuda(),
        # attention_mask=torch.ones(new_input_embeds.size()[:2]).cuda().long(),
        labels=labels,
    ).loss
original_loss

tensor(0.2882, device='cuda:0')

In [8]:
compression_tokens = 4
# compression_tokens = max_new_tokens // 2
# compression_tokens = max_new_tokens // 8
compression_tensor = torch.nn.Parameter(
    torch.rand_like(
        new_input_embeds[:, :compression_tokens, :],
    )
    * model.get_input_embeddings().weight.data.std(),
    requires_grad=True,
)
compressed_inputs_embeds = torch.cat(
    [
        input_ids_embeds.detach(),
        compression_tensor,
        generated_embeds[:, -(max_new_tokens // 2) :, :].detach(),
    ],
    dim=1,
)

In [9]:
compression_tensor.shape, compression_tokens

(torch.Size([1, 4, 1536]), 4)

In [10]:
# compressed_labels = question_input_ids
# comp
question_labels = question_input_ids.clone()
question_labels[0][question_content_mask == 0] = -100
question_labels = question_labels.cuda()
compressed_part = torch.ones(compression_tensor.shape[:2]).long().cuda()

compressed_labels = torch.cat(
    [
        question_labels,
        compressed_part,
        generated_ids_new[:, -(max_new_tokens // 2) :],
    ],
    dim=-1,
)
# compressed_labels

In [11]:
with torch.no_grad():
    compression_loss = model(
        inputs_embeds=compressed_inputs_embeds,
        labels=compressed_labels,
    ).loss

compression_loss

tensor(0.9204, device='cuda:0')

In [18]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Requires Gradient: {param.requires_grad}")

In [126]:
compression_loss.item() <= original_loss.item()

False

In [12]:
epoch_amount = 500
# epoch_amount = 500

optimizer = torch.optim.AdamW([compression_tensor], lr=0.001)
for epoch in range(epoch_amount):
    print(epoch)
    compressed_inputs_embeds = torch.cat(
        [
            input_ids_embeds.detach(),
            compression_tensor,
            generated_embeds[:, -(max_new_tokens // 2) :, :].detach(),
        ],
        dim=1,
    )
    compression_loss = model(
        inputs_embeds=compressed_inputs_embeds,
        labels=compressed_labels,
    ).loss
    print(compression_loss)
    compression_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if compression_loss.item() <= original_loss.item():
        break

0
tensor(0.9204, device='cuda:0', grad_fn=<NllLossBackward0>)
1
tensor(0.8734, device='cuda:0', grad_fn=<NllLossBackward0>)
2
tensor(0.8282, device='cuda:0', grad_fn=<NllLossBackward0>)
3
tensor(0.8105, device='cuda:0', grad_fn=<NllLossBackward0>)
4
tensor(0.7843, device='cuda:0', grad_fn=<NllLossBackward0>)
5
tensor(0.7675, device='cuda:0', grad_fn=<NllLossBackward0>)
6
tensor(0.7508, device='cuda:0', grad_fn=<NllLossBackward0>)
7
tensor(0.7429, device='cuda:0', grad_fn=<NllLossBackward0>)
8
tensor(0.7374, device='cuda:0', grad_fn=<NllLossBackward0>)
9
tensor(0.7302, device='cuda:0', grad_fn=<NllLossBackward0>)
10
tensor(0.7225, device='cuda:0', grad_fn=<NllLossBackward0>)
11
tensor(0.7146, device='cuda:0', grad_fn=<NllLossBackward0>)
12
tensor(0.7066, device='cuda:0', grad_fn=<NllLossBackward0>)
13
tensor(0.6994, device='cuda:0', grad_fn=<NllLossBackward0>)
14
tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward0>)
15
tensor(0.6875, device='cuda:0', grad_fn=<NllLossBackward0>)
16

In [55]:
compression_tensor

Parameter containing:
tensor([[[ 0.1333, -0.0525,  0.0161,  ...,  0.0050,  0.0536,  0.0615],
         [ 0.0280, -0.0860, -0.0381,  ..., -0.0432,  0.0546, -0.0791],
         [-0.0107,  0.0577, -0.0345,  ..., -0.0124,  0.1097,  0.0769],
         ...,
         [ 0.1454,  0.0869,  0.1768,  ...,  0.0386,  0.0551, -0.0129],
         [ 0.1206, -0.0238,  0.0889,  ...,  0.0130,  0.0191,  0.0909],
         [ 0.1462,  0.2155, -0.0516,  ...,  0.1534,  0.2087,  0.1910]]],
       device='cuda:0', requires_grad=True)

In [149]:
new_input_embeds.shape[1]

139

In [ ]:
compression_loss  # / compressed_inputs_embeds.shape[1]

tensor(0.2715, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
original_loss  # / new_input_embeds.shape[1]

tensor(0.2694, device='cuda:0')

In [13]:
with torch.no_grad():

    compressed_inputs_embeds = torch.cat(
        [
            input_ids_embeds.detach(),
            compression_tensor,
            # torch.rand_like(compression_tensor).cuda(),
            generated_embeds[:, -(max_new_tokens // 2) :, :].detach(),
        ],
        dim=1,
    )
    generated_ids_compressed = model.generate(
        inputs_embeds=compressed_inputs_embeds,
        attention_mask=torch.ones(
            compressed_inputs_embeds.shape[:2],
            device="cuda",
        ).long(),
        max_new_tokens=4096,
        # max_new_tokens=5,
        do_sample=False,
        # do_sample=True,
        # temperature=0.6,
        # top_p=0.95,
    )
    # break
generated_result = tokenizer.decode(generated_ids_compressed[-1])
print(generated_result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


 90 degrees, so cos A is positive. Therefore, cos A = 1/√5.

Then, sin A = 2 cos A = 2/√5. Now, tan A is sin A over cos A, so:

tan A = (2/√5) / (1/√5) = 2

Yep, that confirms it. So, tan A is indeed 2. That makes sense because if tan A is 2, then the sides opposite and adjacent to angle A must be in the ratio 2:1. So, if we imagine a right triangle where angle A has opposite side length 2 and adjacent side length 1, the hypotenuse would be √(1² + 2²) = √5. That fits with what we found earlier.

Wait, just to double-check, let's think about the original equation: sin A = 2 cos A. If tan A is 2, then sin A is 2/√5 and cos A is 1/√5, so 2/√5 equals 2*(1/√5), which is 2/√5. Yep, that checks out.

I think that's solid. So, the answer should be 2.

**Final Answer**
The value of $\tan A$ is \boxed{2}.
</think>

Given a right triangle \(ABC\) with \(\angle B = 90^\circ\), we know that \(\sin A = 2 \cos A\). We need to find \(\tan A\).

First, we use the given equation:
\[
\sin A = 2 \cos A
\]

In [14]:
# print()
gold_answer = correct_dataset[dataset_pos]["answer"]
answer = dataset_answer_filter(gold_answer)
model_answer = model_answer_filter(generated_result)
if is_equiv(answer, model_answer):
    correct_items += 1
    print("CORRECT")
else:
    print("WRONG", gold_answer)
    print(generated_result)
compressed_total_len = generated_ids_compressed.shape[1]
print(
    generated_ids_compressed.shape[1],
    compressed_total_len,
    # torch.tensor(generated_ids).shape[1],
)

CORRECT
622 622


In [15]:
(
    compression_tensor.shape[1]
    + generated_embeds[:, -(max_new_tokens // 2) :, :].shape[1]
    + generated_ids_compressed.shape[1]
)

826

In [ ]:
len(
    tokenizer.encode(
        correct_dataset[dataset_pos]["model_answer"],
        add_special_tokens=False,
    )
)

1080